In [1]:
import numpy as np
import keras
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import load_img, img_to_array

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [40]:
import re
from random import shuffle
from glob import glob

input_shape = (224, 224, 3)
train_files = glob('data/cats-dogs/train/*.jpg')
val_files = glob('data/cats-dogs/val/*.jpg')
test_files = glob('data/cats-dogs/test/*.jpg')

def load_image(path, target_size=input_shape[:2]):
    img = load_img(path, target_size=target_size)
    array = img_to_array(img)
    return preprocess_input(array)

# генератор для последовательного чтения обучающих данных с диска
def fit_generator(files, batch_size=64):
    while True:
        shuffle(files)
        for k in range(len(files) // batch_size):
            i = k * batch_size
            j = i + batch_size
            if j > len(files):
                j = - j % len(files)
            x = np.array([load_image(path) for path in files[i:j]])
            y = np.array([(0, 1) if re.match('.*/dog\.\d', path) else (1, 0) for path in files[i:j]])
            yield (x, y)

def load_validation(files=val_files):
    x = np.array([load_image(path) for path in files])
    y = np.array([(0, 1) if re.match('.*/dog\.\d', path) else (1, 0) for path in files])
    return (x, y)

def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])

In [3]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in base_model.layers:
    layer.trainable = False

In [4]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [46]:
x = base_model.layers[-2].output
x = keras.layers.Conv2D(50, (1, 1), kernel_initializer='random_uniform', activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Conv2D(50, (3, 3), kernel_initializer='random_uniform', activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2, kernel_initializer='random_uniform', activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=x)

In [47]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [7]:
#model.load_weights('mymodel')
validation_data = load_validation()

In [32]:
keras.optimizers.Adam?

In [48]:
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

shuffle(train_files)
#model.load_weights('mymodel')
model.fit_generator(
    fit_generator(train_files),
    steps_per_epoch=50,
    epochs=30,
    validation_data=validation_data)
#model.save('resnet-last'.format(ind))

Epoch 1/30
50/50 [==============================] - 32s 634ms/step - loss: 0.5679 - acc: 0.7028 - val_loss: 0.2326 - val_acc: 0.9490
Epoch 2/30
50/50 [==============================] - 31s 620ms/step - loss: 0.2113 - acc: 0.9227 - val_loss: 0.0756 - val_acc: 0.9785
Epoch 3/30
50/50 [==============================] - 31s 627ms/step - loss: 0.1335 - acc: 0.9480 - val_loss: 0.0697 - val_acc: 0.9775
Epoch 4/30
50/50 [==============================] - 31s 623ms/step - loss: 0.0989 - acc: 0.9620 - val_loss: 0.0759 - val_acc: 0.9775
Epoch 5/30
50/50 [==============================] - 31s 620ms/step - loss: 0.0794 - acc: 0.9692 - val_loss: 0.0646 - val_acc: 0.9800
Epoch 6/30
50/50 [==============================] - 32s 638ms/step - loss: 0.0756 - acc: 0.9725 - val_loss: 0.0659 - val_acc: 0.9805
Epoch 7/30
50/50 [==============================] - 31s 628ms/step - loss: 0.0851 - acc: 0.9670 - val_loss: 0.0681 - val_acc: 0.9805
Epoch 8/30
50/50 [==============================] - 31s 623ms/step - 

In [49]:
model.save('mymodel')

In [ ]:
%%time
pred = model.predict_generator(predict_generator(test_files), len(test_files), max_queue_size=500)
pred

In [ ]:
p = np.argmax(pred, axis=1)
with open('submit.txt', 'w') as dst:
    dst.write('id,label\n')
    for path, score in zip(test_files, p):
        dst.write('%s,%f\n' % (re.search('(\d+)', path).group(0), score))

In [ ]:
# LogLoss = 1.04979